In [2]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-04-30 15:58:18--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  37.3MB/s    in 5m 7s   

2022-04-30 16:03:25 (21.0 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-04-30 16:03:57--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [3]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.InceptionV3(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

96124928/96112376 [==============================] - 1s 0us/step


In [4]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.inception_v3.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))


49152/35363 [=========================================] - 0s 0us/step
1%, Top 1:0.742 Top 5:0.934
2%, Top 1:0.756 Top 5:0.927
3%, Top 1:0.7726666666666666 Top 5:0.9253333333333333
4%, Top 1:0.763 Top 5:0.9215
5%, Top 1:0.7736 Top 5:0.928
6%, Top 1:0.7703333333333333 Top 5:0.925
7%, Top 1:0.7668571428571429 Top 5:0.9271428571428572
8%, Top 1:0.7665 Top 5:0.928
9%, Top 1:0.7677777777777778 Top 5:0.9275555555555556
10%, Top 1:0.7682 Top 5:0.928
11%, Top 1:0.7694545454545455 Top 5:0.9274545454545454
12%, Top 1:0.7696666666666667 Top 5:0.9285
13%, Top 1:0.7710769230769231 Top 5:0.9283076923076923
14%, Top 1:0.7704285714285715 Top 5:0.9288571428571428
15%, Top 1:0.7710666666666667 Top 5:0.9288
16%, Top 1:0.77025 Top 5:0.92875
17%, Top 1:0.7707058823529411 Top 5:0.9290588235294117
18%, Top 1:0.7697777777777778 Top 5:0.9287777777777778
19%, Top 1:0.7698947368421053 Top 5:0.9290526315789474
20%, Top 1:0.7705 Top 5:0.9282
21%, Top 1:0.7699047619047619 Top 5:0.929047619047619
22%, Top 1:0.7690909

In [5]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:76.276%
Top 5 Accuracy:93.03200000000001%
